# Pre requisites

Install and import required packages

In [1]:
!pip install boto3
!pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/18/c6/1c9c86e51f42e3a601aab9c6d37c9b3c1aa859deb21be7e29938f008b499/boto3-1.29.4-py3-none-any.whl.metadata
  Obtaining dependency information for botocore<1.33.0,>=1.32.4 from https://files.pythonhosted.org/packages/e7/27/c0447944668099c4eb374d2330579ed3aacb27e07a1fd5ca13040629bdd6/botocore-1.32.4-py3-none-any.whl.metadata
  Obtaining dependency information for s3transfer<0.8.0,>=0.7.0 from https://files.pythonhosted.org/packages/5a/4b/fec9ce18f8874a96c5061422625ba86c3ee1e6587ccd92ff9f5bf7bd91b2/s3transfer-0.7.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/135.8 kB ? eta -:--:--
   --- ------------------------------------ 10.2/135.8 kB ? eta -:--:--
   ----------- --------------------------- 41.0/135.8 kB 653.6 kB/s eta 0:00:01
   -------------------- ------------------ 71.7/135.8 kB 558.5 kB/

In [2]:
import boto3
from pathlib import Path
from botocore import UNSIGNED
from botocore.client import Config
from tqdm.notebook import tqdm

# Download files from s3 bucket

Get a list of all files in the bucket and download into respective folders within the colab runtime

In [3]:
def get_file_folders(s3_client, bucket_name, prefix=""):
    file_names = []
    folders = []

    default_kwargs = {
        "Bucket": bucket_name,
        "Prefix": prefix
    }
    next_token = ""

    while next_token is not None:
        updated_kwargs = default_kwargs.copy()
        if next_token != "":
            updated_kwargs["ContinuationToken"] = next_token

        response = s3_client.list_objects_v2(**updated_kwargs)
        contents = response.get("Contents")

        for result in contents:
            key = result.get("Key")
            if key[-1] == "/":
                folders.append(key)
            else:
                file_names.append(key)

        next_token = response.get("NextContinuationToken")

    return file_names, folders

In [4]:
def download_files(s3_client, bucket_name, local_path, file_names, folders):
    local_path = Path(local_path)

    for folder in tqdm(folders):
        folder_path = Path.joinpath(local_path, folder)
				# Create all folders in the path
        folder_path.mkdir(parents=True, exist_ok=True)

    for file_name in tqdm(file_names):
        file_path = Path.joinpath(local_path, file_name)
				# Create folder for parent directory
        file_path.parent.mkdir(parents=True, exist_ok=True)
        s3_client.download_file(
            bucket_name,
            file_name,
            str(file_path)
        )

In [5]:
client = boto3.client('s3', config=Config(signature_version=UNSIGNED))
file_names, folders = get_file_folders(client, 'cgiar-crop-damage-classification-challenge')

In [6]:
download_files(
        client,
        'cgiar-crop-damage-classification-challenge',
        r"C:\Users\SameerAhamed\Downloads\cgiar-crop-damage-classification-challenge",
        file_names,
        folders
    )

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]